<a href="https://colab.research.google.com/github/Diegoarmando24/Practica-2/blob/main/Ejercicio_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy.linalg import cond  # Solo se usa para calcular el número de condición (permitido)

# ==========================================
# FUNCIONES AUXILIARES PARA RESOLUCIÓN LU
# ==========================================

def SustitucionDelante(Mat, b):
    """
    Realiza sustitución hacia adelante.
    Matriz Mat debe ser triangular inferior.
    """
    n = Mat.shape[0]
    x = np.zeros(n)
    for i in range(n):
        SumCum = sum(Mat[i, j] * x[j] for j in range(i))
        x[i] = (b[i] - SumCum) / Mat[i, i]
    return x

def SustitucionAtras(Mat, b):
    """
    Realiza sustitución hacia atrás.
    Matriz Mat debe ser triangular superior.
    """
    n = Mat.shape[0]
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        SumCum = sum(Mat[i, j] * x[j] for j in range(i + 1, n))
        x[i] = (b[i] - SumCum) / Mat[i, i]
    return x

def LU(A):
    """
    Factorización LU sin pivoteo.
    Devuelve matrices L (inferior) y U (superior) tal que A = LU.
    """
    U = np.copy(A)
    L = np.eye(A.shape[0])
    for i in range(A.shape[0]):
        Li = np.eye(A.shape[0])
        for j in range(i + 1, A.shape[0]):
            Li[j, i] = -U[j, i] / U[i, i]
            L[j, i] = U[j, i] / U[i, i]
        U = Li @ U
    return L, U

def SolverLU(A, b):
    """
    Resuelve Ax = b mediante factorización LU (sin pivoteo).
    """
    L, U = LU(A)
    y = SustitucionDelante(L, b)
    x = SustitucionAtras(U, y)
    return x

# ================================================
# FUNCIONES PARA FACTORIZACIÓN LU CON PIVOTEO
# ================================================

def Permutaciones_Parcial(A, b):
    """
    Aplica pivoteo parcial: intercambia filas para maximizar elementos pivote.
    """
    U = np.copy(A)
    x = np.copy(b)
    P = np.eye(A.shape[0])
    for j in range(A.shape[0]):
        k = np.argmax(np.abs(U[j:, j])) + j
        U[[j, k]] = U[[k, j]]
        P[[j, k]] = P[[k, j]]
        x[[j, k]] = x[[k, j]]
    return P, U, x

def Solver_LU_Pivot_Partial(A, b):
    """
    Resuelve Ax = b usando LU con pivoteo parcial.
    """
    P, Ap, bp = Permutaciones_Parcial(A, b)
    return SolverLU(Ap, bp)

def Permutaciones_Totales(A, b):
    """
    Aplica pivoteo total: intercambia filas y columnas para maximizar pivotes.
    """
    n = A.shape[0]
    U = np.copy(A)
    x = np.copy(b)
    P = np.eye(n)
    Q = np.eye(n)
    for k in range(n - 1):
        max_row, max_col = np.unravel_index(np.argmax(np.abs(U[k:, k:])), (n - k, n - k))
        max_row += k
        max_col += k
        U[[k, max_row]] = U[[max_row, k]]
        P[[k, max_row]] = P[[max_row, k]]
        x[[k, max_row]] = x[[max_row, k]]
        U[:, [k, max_col]] = U[:, [max_col, k]]
        Q[:, [k, max_col]] = Q[:, [max_col, k]]
    return P, Q, U, x

def Solver_LU_Pivot_Total(A, b):
    """
    Resuelve Ax = b usando LU con pivoteo total.
    """
    P, Q, A_g, b_g = Permutaciones_Totales(A, b)
    x = SolverLU(A_g, b_g)
    return Q @ x

# ================================================
# PRUEBA DEL EJERCICIO 21 - INCISO (a)
# ================================================

A = np.array([[4, -1, 3],
              [-8, 4, -7],
              [12, 1, 8]], dtype=float)
b = np.array([-8, 19, -19], dtype=float)

# Resolver con distintos métodos
x_sin_pivoteo = SolverLU(A, b)
x_piv_parcial = Solver_LU_Pivot_Partial(A, b)
x_piv_total = Solver_LU_Pivot_Total(A, b)

# Condición de la matriz A
cond_A = cond(A)

# Mostrar resultados
print("Inciso (a):")
print("LU sin pivoteo:        ", x_sin_pivoteo)
print("LU con pivoteo parcial:", x_piv_parcial)
print("LU con pivoteo total:  ", x_piv_total)
print("Número de condición de:", cond_A)

# ================================================
# PRUEBA DEL EJERCICIO 21 - INCISO (b)
# ================================================

A2 = np.array([[1, 4, -2, 1],
               [-2, -4, -3, 1],
               [1, 16, -17, 9],
               [2, 4, -9, -3]], dtype=float)
b2 = np.array([3.5, -2.5, 15, 10.5], dtype=float)

x2_LU = SolverLU(A2, b2)
x2_Partial = Solver_LU_Pivot_Partial(A2, b2)
x2_Total = Solver_LU_Pivot_Total(A2, b2)
cond2 = cond(A2)

print("Inciso (b):")
print("LU sin pivoteo:        ", x2_LU)
print("LU con pivoteo parcial:", x2_Partial)
print("LU con pivoteo total:  ", x2_Total)
print("Número de condición:   ", cond2)
print()

# ================================================
# PRUEBA DEL EJERCICIO 21 - INCISO (c)
# ================================================

A3 = np.array([[4, 5, -1, 2, 3],
               [12, 13, 0, 10, 3],
               [-8, -8, 5, -11, 4],
               [16, 18, -7, 20, 4],
               [-4, -9, 31, -31, -1]], dtype=float)
b3 = np.array([34, 93, -33, 131, -58], dtype=float)

x3_LU = SolverLU(A3, b3)
x3_Partial = Solver_LU_Pivot_Partial(A3, b3)
x3_Total = Solver_LU_Pivot_Total(A3, b3)
cond3 = cond(A3)

print("Inciso (c):")
print("LU sin pivoteo:        ", x3_LU)
print("LU con pivoteo parcial:", x3_Partial)
print("LU con pivoteo total:  ", x3_Total)
print("Número de condición:   ", cond3)


Inciso (a):
LU sin pivoteo:         [-1.  1. -1.]
LU con pivoteo parcial: [-1.  1. -1.]
LU con pivoteo total:   [-1.  1. -1.]
Número de condición de: 173.7410040078959
Inciso (b):
LU sin pivoteo:         [-0.5  1.  -0.5 -1. ]
LU con pivoteo parcial: [-0.5  1.  -0.5 -1. ]
LU con pivoteo total:   [-0.5  1.  -0.5 -1. ]
Número de condición:    287.3867135503838

Inciso (c):
LU sin pivoteo:         [1. 2. 3. 4. 5.]
LU con pivoteo parcial: [1. 2. 3. 4. 5.]
LU con pivoteo total:   [1. 2. 3. 4. 5.]
Número de condición:    384.2113665970671
